In [1]:
#Initializes plotting library and functions for 3D scatter plots 
from pyspark.ml.feature import VectorAssembler
from sklearn.datasets import make_blobs
from sklearn.datasets import make_gaussian_quantiles
from sklearn.datasets import make_classification, make_regression
from sklearn.externals import six
import pandas as pd
import numpy as np
import argparse
import json
import re
import os
import sys
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()

def rename_columns(df, prefix='x'):
    """
    Rename the columns of a dataframe to have X in front of them

    :param df: data frame we're operating on
    :param prefix: the prefix string
    """
    df = df.copy()
    df.columns = [prefix + str(i) for i in df.columns]
    return df



AttributeError: module 'numpy' has no attribute 'core'

In [76]:
# create an artificial dataset with 3 clusters
X, Y = make_classification(n_samples=100, n_classes=4, n_features=12, n_redundant=0, n_informative=12,
                             scale=1000, n_clusters_per_class=1)
df = pd.DataFrame(X)
# ensure all values are positive (this is needed for our customer 360 use-case)
df = df.abs()
# rename X columns
df = rename_columns(df)
# and add the Y
df['y'] = Y

# split df into cluster groups
grouped = df.groupby(['y'], sort=True)

# compute sums for every column in every group
sums = grouped.sum()
sums


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
y,,,,,,,,,,,,
0,43917.902717,35270.451789,43395.553830,43264.556543,44798.047771,43170.541005,34978.731256,46451.970844,39157.214993,47472.228340,52985.400023,34268.316353
1,49737.117164,45376.954260,33026.290298,59482.369256,42730.829847,38790.301033,52704.949106,48227.990196,55560.812577,40686.422554,43150.751511,45556.518766
2,54915.653042,45577.167943,36426.522912,44407.995024,32359.063697,41894.707387,57660.106809,53733.010771,65061.040075,30963.331417,31429.731367,44558.999544
3,42181.965937,44906.545233,44438.018456,45166.143468,29628.328743,31665.603171,37991.480200,53671.437781,57576.106698,40516.897999,52338.713560,44832.938070


In [2]:
import numpy as np

from bokeh.util.browser import view
from bokeh.document import Document
from bokeh.embed import file_html
from bokeh.models.glyphs import ImageURL
from bokeh.models import ColumnDataSource, Range1d, Plot, LinearAxis, Grid
from bokeh.resources import INLINE

url = "http://bokeh.pydata.org/en/latest/_static/images/logo.png"
N = 5

source = ColumnDataSource(dict(
    url = [url]*N,
    x1  = np.linspace(  0, 150, N),
    y1  = np.linspace(  0, 150, N),
    w1  = np.linspace( 10,  50, N),
    h1  = np.linspace( 10,  50, N),
    x2  = np.linspace(-50, 150, N),
    y2  = np.linspace(  0, 200, N),
))

xdr = Range1d(start=-100, end=200)
ydr = Range1d(start=-100, end=200)

plot = Plot(x_range=xdr, y_range=ydr)
plot.title.text = "ImageURL"
plot.toolbar_location = None

image1 = ImageURL(url="url", x="x1", y="y1", w="w1", h="h1", anchor="center", global_alpha=0.2)
plot.add_glyph(source, image1)

image2 = ImageURL(url="url", x="x2", y="y2", w=20, h=20, anchor="top_left")
plot.add_glyph(source, image2)

image3 = ImageURL(url=dict(value=url), x=200, y=-100, anchor="bottom_right")
plot.add_glyph(source, image3)

xaxis = LinearAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis,'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

doc = Document( )
doc.add_root(plot)

if __name__ == "__main__":
    doc.validate()
    filename = "image_url.html"
    with open(filename, "w") as f:
        f.write(file_html(doc, INLINE, "Image URL Example"))
    print("Wrote %s" % filename)
    view(filename)


Wrote image_url.html
